<h1>
    <p align="center">
        <b>
            <font color='gold'>
        Análise Exploratória: Incêndios Florestais Brasileiros 🔥🌳
            </font>
        </b>
    </p>
</h1>

# <font color='gold'> **Objetivos da Análise** 🎯 </font>
---

Nesse projeto, realizaremos uma Analise Exploratória de Dados (EDA) em um [banco de dados](https://www.kaggle.com/datasets/gustavomodelli/forest-fires-in-brazil) que contém uma série histórica de incêndios florestais no Brasil em diferentes estados. Mais especificamente, desse projeto iremos:

- 🎲 Realizar uma análise descritiva das características principais dos dados, como suas dimensões, seus registros e tipos de dados, valores nulos e outliers, etc;

- 🧹 Limpar e tratar os dados;

- 📊 Utilizar de visualizações gráficas para geração de insights e análises mais aprofundadas acerca dos incêndios e suas características.

Como questões centrais que abordaremos nessa pesquisa, temos:

>***1. Quais os estados e regiões mais afetados por incêndios durante toda a série histórica?***
>
>***2. Qual a década com mais registros de incêndios no Brasil?***
>
>***3. Durante qual período de qual governante brasileiro tivemos mais incêndios no Brasil? e qual o menor valor?***

Entre outras questões que serão abordadas durante o projeto.

# <font color='gold'>***Importando Pacotes e Conhecendo Dados*** 📦🎲</font>
---

In [42]:
# Importando bibliotecas necessárias para a análise:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Definindo parâmetros:

# Setando estilo escuro do grid para os gráficos.
sns.set_theme(context='notebook', style='darkgrid')

# Utilizando a biblioteca warnings para retirar avisos da execução do código
# para uma visualização mais limpa.
warnings.filterwarnings('ignore')

# Mostrando versões utilizadas dos pacotes:
print('Pacotes Prontos! Versões Utilizadas:\n')
print(f'Pandas      --> {pd.__version__}')
print(f'Numpy       --> {np.__version__}')
print(f'Matplotlib  --> {matplotlib.__version__}')
print(f'Seaborn     --> {sns.__version__}')

Pacotes Prontos! Versões Utilizadas:

Pandas      --> 1.4.2
Numpy       --> 1.21.5
Matplotlib  --> 3.6.2
Seaborn     --> 0.11.2


In [43]:
# Criando o Dataframe:
data = pd.read_csv('data/amazon.csv', sep=',', encoding='latin-1')

<font color='gold'> ⚠️**Nota** </font>

>A seleção do encoding como [latin-1](https://www.ic.unicamp.br/~stolfi/EXPORT/www/ISO-8859-1-Encoding.html) foi utilizada pois existem caracteres especiais nos dados, assim, a definição do enconding específico foi necessária (em vez do padrão UTF-8).

In [44]:
# Visualizando primeiros 5 registros dos dados:
data.head()

,year,state,month,number,date
0,1998,Acre,Janeiro,0.0,1998-01-01
1,1999,Acre,Janeiro,0.0,1999-01-01
2,2000,Acre,Janeiro,0.0,2000-01-01
3,2001,Acre,Janeiro,0.0,2001-01-01
4,2002,Acre,Janeiro,0.0,2002-01-01


In [45]:
# Visualizando uma sample aleatória dos dados:
data.sample(5)

,year,state,month,number,date
1122,2003,Bahia,Setembro,8.847,2003-01-01
4440,2015,Rio,Julho,74.000,2015-01-01
6383,2006,Tocantins,Setembro,2.979,2006-01-01
2266,2012,Maranhao,Junho,915.000,2012-01-01
4744,2000,Rio,Novembro,9.000,2000-01-01


Para termos uma compreensão clara de nossos dados, iremos criar um dicionário de variáveis contendo as colunas do Dataset e a descrição de seu conteúdo:

| Variável   | Conteúdo                                 |
|------------|------------------------------------------|
| `year`     | Ano em que o incêndio florestal ocorreu  |
| `state`    | Estado brasileiro                        |
| `month`    | Mês em que o incêndio ocorreu            |
| `number`   | Quantidade de incêndios reportados       |
| `date`     | Data em que o incêndio foi reportado     |

Com alguns dos registros aparentes, uma característica notável é a aparente duplicata nas colunas. 

- A coluna `year`, que representa o ano em que o incêndio ocorreu e a coluna `date` que informa a data em que o incêndio foi reportado, aparentam possuir os mesmos dados:

    - Quando a coluna year apresenta o ano de 1998, a coluna date informa a data 1998-01-01, e esse padrão da coluna date apenas repetir a coluna year com a adição das datas "1 de Janeiro" se mostra presente em todo dataset como mostrado pelo uso do comando `data.sample(5)`.

Dessa forma, iremos arranjar essas informações de uma maneira diferente:

- Substituir os valores da coluna `month` por numéricos ('Janeiro' = 1, 'Fevereiro' = 2, etc);

- Utilizar a coluna `date` para mostrar o ano e o mês (agora em formato numérico) do registro;

- Transformar essa nova coluna `date` para datetime.

Como outros ajustes, também iremos renomear as colunas para que tenham a primeira letra maiúscula e também, renomear algumas colunas. Também iremos inserir algumas novas colunas para fortalecer as análises, tendo essas etapas ocorrendo na etapa de limpeza e tratamento dos dados.

Para uma visão mais completa das dimensões dos dados que estamos trabalhando, vamos checar o shape do Dataset:

In [46]:
# Checando dimensões dos dados:
data.shape

(6454, 5)

Temos ***6454 registros*** em ***5 variáveis***.

Também se faz necessária a checagem do tipo de dados dessas variáveis:

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    6454 non-null   int64  
 1   state   6454 non-null   object 
 2   month   6454 non-null   object 
 3   number  6454 non-null   float64
 4   date    6454 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 252.2+ KB


Com essa visualização, temos algumas informações importantes sobre os dados:

- O dataset não possui null values;

- As colunas `year`, `month` e `date` serão unidas numa só coluna datetime;

- Os dados alocam pouca quantidade de memória (cerca de 250 Kb).

Para coletarmos mais informações sobre possíveis outliers e também sobre a distribuição dos dados, utilizaremos o método `.describe()`:

In [99]:
# Descrição estatística dos dados (colunas numéricas):
data.describe().T

,count,mean,std,min,25%,50%,75%,max
year,6454.0,2007.461729,5.746654,1998.0,2002.0,2007.0,2012.0,2017.0
number,6454.0,108.293163,190.812242,0.0,3.0,24.0,113.0,998.0


Com a descrição estatística das colunas numéricas do dataset, podemos notar:

- Até 75% dos dados (terceiro quantil) relacionados ao número de incêndios relatados vão até 113, já o valor máximo é de 998, um outlier claro.

Nesse caso, outliers são esperados como "normais", pois se tratando de número de focos de incêndios florestais num país continental como o Brasil, não existe uma regra ou uma lógica exata que dita o intervalo desses números, tendo muitas variávies que os influenciam (ação humana mais intensa ou menos intensa numa determinada região num determinado período de tempo, uma estação do ano muito mais fria ou quente do que o normal, etc).

Sendo assim, os registros de número de focos de incêndio não serão alterados, pois buscamos ter uma visão completa desses dados.

Também iremos coletar informações sobre as colunas categóricas do dataset:

In [98]:
# Descrição estatística dos dados (colunas categóricas):
data.describe(exclude='number').T

,count,unique,top,freq
state,6454,23,Rio,717
month,6454,12,Janeiro,541
date,6454,20,1998-01-01,324


Com isso, temos a informação de que o estado que mais vezes consta no dataset é o Rio de Janeiro, com 717 registros diferentes de focos de incêndio.

Também temos a informação de que o mês mais comum nos dados é Janeiro.

É importante termos em mente que isso não necessariamente significa que o estado do Rio de Janeiro e o mês de Janeiro são os que mais possuem focos de incêndio no geral. Apenas estamos indicando que essas duas informações são as mais presentes nos dados, sem considerarmos a dimensão e quantidade dos focos de incêndio atribuidos a eles.

# <font color='gold'> ***Limpeza e Tratamento*** 🗃️ </font>